In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import quad
import math

# 1. Define a function that returns the sum of n Gaussians
def sum_of_n_gaussians_factory(n):
    """
    Returns a function f(E, *params) which is the sum of n Gaussians.
    params: (A1, E01, sigma1, A2, E02, sigma2, ..., An, E0n, sigman).
    """
    def multi_gaussian_n(E, *params):
        total = np.zeros_like(E, dtype=float)
        for i in range(n):
            A     = params[3*i + 0]
            E0    = params[3*i + 1]
            sigma = params[3*i + 2]
            total += A * np.exp(-((E - E0)**2) / (2*sigma**2))
        return total
    return multi_gaussian_n

def single_gaussian(E, A, E0, sigma):
    """ Single Gaussian in energy space """
    return A * np.exp(-((E - E0)**2) / (2*sigma**2))

def main():
    # -------------------------------------------------------------------------
    # A) Load data from .mca file (example: '5 shekels ref.mca')
    # -------------------------------------------------------------------------
    file_path = r"C:\Users\David1\Downloads\Nickel 100um tran.mca"

    with open(file_path, 'rb') as file:
        file_content = file.read().decode('latin1', errors='ignore')

    data_start_index = file_content.find('<<DATA>>') + len('<<DATA>>')
    data_string_full = file_content[data_start_index:]
    counts_full = [int(v) for v in data_string_full.split() if v.isdigit()]

    # Keep only the first 8192 channels (typical for many MCAs)
    counts_trimmed = counts_full[:8192]

    # -------------------------------------------------------------------------
    # B) Convert all channels (0..8191) to energy
    #    E(keV) = channel * 0.003798 - 0.19
    # -------------------------------------------------------------------------
    channels = np.arange(8192)
    energy_all = channels * 0.003798 - 0.19

    # Convert counts to float array for arithmetic
    counts_all = np.array(counts_trimmed, dtype=float)

    # -------------------------------------------------------------------------
    # C) Define an energy cut range: [energy_min, energy_max]
    # -------------------------------------------------------------------------
    energy_min = 7   # keV (adjust to your region of interest)
    energy_max = 9   # keV (adjust to your region of interest)

    # Create a boolean mask of points within this energy range
    mask = (energy_all >= energy_min) & (energy_all <= energy_max)

    # Extract the subset for the fit
    energy_cut = energy_all[mask]
    counts_cut = counts_all[mask]

    # -------------------------------------------------------------------------
    # D) Convert raw counts to counts/hour (optional)
    # -------------------------------------------------------------------------
    measurement_time_s = 151123.992
    counts_cut_tran = counts_cut * (3600.0 / measurement_time_s)

    # -------------------------------------------------------------------------
    # E) Choose how many Gaussians (n_peaks) you want to fit
    # -------------------------------------------------------------------------
    n_peaks = 2

    # Create the multi-Gaussian function for n_peaks
    multi_gaussian = sum_of_n_gaussians_factory(n_peaks)

    # -------------------------------------------------------------------------
    # F) Provide initial guesses for each of the n Gaussians:
    #    [A1, E01, sigma1, A2, E02, sigma2, A3, E03, sigma3, A4, E04, sigma4]
    # -------------------------------------------------------------------------
    initial_params = [
        0.07,  7.47975716,   10,
        0.04,  8.26267688,   10
    ]

    # -------------------------------------------------------------------------
    # G) Define parameter bounds: A >= 0, E0 in [energy_min, energy_max], sigma >= 0
    # -------------------------------------------------------------------------
    lower_bounds = []
    upper_bounds = []
    for i in range(n_peaks):
        lower_bounds += [0.0,    energy_min, 0.0]
        upper_bounds += [np.inf, energy_max, np.inf]
    bounds = (lower_bounds, upper_bounds)

    # -------------------------------------------------------------------------
    # H) Fit the data in energy space
    # -------------------------------------------------------------------------
    try:
        popt, pcov = curve_fit(
            multi_gaussian,
            energy_cut,
            counts_cut_tran,
            p0=initial_params,
            bounds=bounds,
            maxfev=10000
        )
        fit_successful = True
    except RuntimeError as e:
        print("Fit did not converge:", e)
        popt, pcov = None, None
        fit_successful = False

    # -------------------------------------------------------------------------
    # I) If fit is successful, print parameters
    # -------------------------------------------------------------------------
    if fit_successful:
        perr = np.sqrt(np.diag(pcov))
        print("==== Fitted Parameters (Energy Space) ====")
        for i in range(n_peaks):
            A_val     = popt[3*i + 0]
            E0_val    = popt[3*i + 1]
            sigma_val = popt[3*i + 2]
            A_err     = perr[3*i + 0]
            E0_err    = perr[3*i + 1]
            sigma_err = perr[3*i + 2]
            print(f"Peak #{i+1}:")
            print(f"  A     = {A_val:.2f} ± {A_err:.2f}  (counts/hour at peak)")
            print(f"  E0    = {E0_val:.3f} ± {E0_err:.3f} (keV)")
            print(f"  sigma = {sigma_val:.3f} ± {sigma_err:.3f} (keV)")

        # ---------------------------------------------------------------------
        # J) Compute areas for each fitted Gaussian
        # ---------------------------------------------------------------------
        print("\n==== Area Analysis for Each Gaussian ====")
        two_pi_sqrt = math.sqrt(2.0 * math.pi)

        for i in range(n_peaks):
            Ai, E0i, si = popt[3*i], popt[3*i+1], popt[3*i+2]
            dAi, dE0i, dsi = perr[3*i], perr[3*i+1], perr[3*i+2]

            # (A) Numerical integration in [energy_min, energy_max]
            def gauss_i(E):
                return single_gaussian(E, Ai, E0i, si)
            partial_area_i, partial_area_err_i = quad(gauss_i, energy_min, energy_max)

            # (B) Theoretical total area (A * sqrt(2π) * sigma)
            area_theory_i = Ai * two_pi_sqrt * si
            darea_theory_i = np.sqrt(
                (dAi * two_pi_sqrt * si)**2 +
                (Ai  * two_pi_sqrt * dsi)**2
            )

            print(f"\nPeak #{i+1}:")
            print(f"  Numerical Area in [{energy_min}, {energy_max}] = "
                  f"{partial_area_i:.10f} ± {partial_area_err_i:.10e} (counts/hour·keV)")
            print(f"  Theoretical Full Area (±∞)      = "
                  f"{area_theory_i:.10f} ± {darea_theory_i:.10f} (counts/hour·keV)")

        # ----- Optional: total area under the sum of n Gaussians over the cut
        def total_model(E):
            return multi_gaussian(E, *popt)

        total_area, total_err = quad(total_model, energy_min, energy_max)
        print(f"\nTotal partial area (sum of all {n_peaks} Gaussians) in "
              f"[{energy_min}, {energy_max}] = {total_area:.2f} ± {total_err:.2e} "
              "(counts/hour·keV)")

        # ---------------------------------------------------------------------
        # K) Plot the data and the fit in energy space
        # ---------------------------------------------------------------------
        fig, ax = plt.subplots(figsize=(10,6))

        # 1) Plot the data (energy vs. counts/hour)
        ax.plot(energy_cut, counts_cut_tran, label='Data (Cut)')

        # 2) Plot the sum of all Gaussians
        fitted_y = multi_gaussian(energy_cut, *popt)
        ax.plot(energy_cut, fitted_y, 'r-', label=f'Sum of {n_peaks} Gaussians')

        # 3) Optionally, plot each individual Gaussian
        for i in range(n_peaks):
            Ai, E0i, si = popt[3*i], popt[3*i + 1], popt[3*i + 2]
            single_fit_y = single_gaussian(energy_cut, Ai, E0i, si)
            ax.plot(energy_cut, single_fit_y, '--')

        # 4) Draw vertical lines at known XRF lines (e.g., Cu Kα, Cu Kβ, Ni Kα, Ni Kβ)
        ax.axvline(x=7.47975716, color='red',    linestyle='--', linewidth=2, label='Ni Ka')
        ax.axvline(x=8.26267688, color='green', linestyle='--', linewidth=2, label='Ni Kb')
        #ax.axvline(x=7.477, color='orange',  linestyle='--', linewidth=2, label='Ni Ka')
        #ax.axvline(x=8.264, color='purple', linestyle='--', linewidth=2, label='Ni Kb')

        ax.set_xlabel('Energy (keV)')
        ax.set_ylabel('Counts per hour')
        # ax.set_title(f'Fitting {n_peaks} Gaussians in [{energy_min} keV, {energy_max} keV]')
        ax.grid(True)
        ax.legend()
        plt.show()
    else:
        print("Fitting failed. No results to show.")

if __name__ == "__main__":
    main()
